# Fitting a black-box function
## Simplex method

Given:
- a set of *n* $(x_i,y_i)$ pairs of experimentaly measured values
- a function $y = f(\vec p,x)$ that should model this set of data by adjusting a set of $m$ parameters $\vec p$ (i.e., p is a vector)

Find:
- parameters $\vec p$ that results in the best fit to the data

We assume that the function $f(\vec p,x)$ cannot be represented in a form that allows finding parameters $\vec p$ by direct methods such as polynomial regression. In that case the parameters $p$ can be found by trial-and-compute method. In a most straightforward scenario, one will pick a range of change for each parameter, divide that range by large number, and compute the value of the error function for each combination of parameters:

$$efun\left (\vec p \right ) = \sum_{i=1}^{n} {\left | y_i - f(\vec p,x_i) \right |^2}$$

This direct method is time-consuming - for $m$ marameters where each can take N values the number of combinations to test is $m^N$, and in most cases that number is just too large to compute.<p>

The problem is equivalent of finding a minimum of a function $efun$. We will consider here a relatively simple linear strategy of finding the minimum of an arbitrary smooth function $efun$ called *Simplex*.<p>

The process is iterative. To begin, a guess, a set of initial parameters $\vec p$ are given along with corresponding values $\vec {\delta p}$. The latter values are the initial guess estimated "precision" for each parameter. The program builds a *simplex* in the parameter space - the simplest shape for the $m$-dimensional space, i.e. a polygon with $m+1$ verteces $v$:
$$
v_0 = (p_1, p_2, p_3, p_4, ...,p_m) \\
v_1 = (p_1 + dp_1, p_2, p_3, ...) \\
v_2 = (p_1, p_2 + dp_2, p_3, ...) \\
v_3 = (p_1, p_2, p_3 + dp_3, ...) \\
... \\
v_m = {p_1, p_2, ..., p_m + dp_m}
$$
In each Iteration<p>
1. Call function for each vertex of the simplex: $efun \left(\vec v_j \right )$, and store function values $\chi_j$ for each vertex<br>
2. Find the worst (highest $\chi$) and best (smallest $\chi$) verteces. This will give you general direction of the gradient of the function, i.e., the direction to take to get to minimum.<br>
3. Refect the worst vertex to the other side of the best vertex, i.e., try to get closer to minimum.<br>
4. Calculate $\chi$-value of that new point.<br>
    4a. If new $\chi$ value is even better than the best point $\chi$, move that new point twice further from the previous best point. In effect this increases the size of the simplex. If this point's $\chi$ is still better than the previous best point, replace the old vertex with the new one in simplex and proceed to next iteration (step 1). This will increase the simplex size in the direction of minimum making next step toward minimum longer<br>
    4b. Otherwise: set the singly-reflected point as a new vertex and go to step 1<br>
    4c. If the newe reflected point is worse than its $\chi$ before reflection - then we apparently overshot. Instead of reflection, move thad previously worst vertex twice closer to the best point and go to step 1. This will effectively make the simplex size in this direction smaller.<p>
        
It is important to provide an escape from that loop once the desired precision is achieved. There are different options that can be used to break the loop:<p>
1. Break if the $\chi$-value of the best vertex is below certain value. This, however, may be not  good strategy as usually we do not know what the lowest value of the $\chi$ that can be achieved.<br>
2. Break if simplex becomes very small in $\chi$ space. I.e., $\chi_{worst} - \chi_{best}$ is smaller than some value.<br>
3. Break if the size of simplex in $\chi$ space is much smaller than the best $\chi$. I.e., when $\left(\chi_{worst} - \chi_{best} \right)\chi_{best}$ is smaller than some value. This is perhaps the best criterion for fitting curves when $\chi$ value is not expected to reach 0. It will not work well if one fits theoretically calculated curve with actual function that wqas used to generate the data as $\chi$ will culminate in 0.<p>

The Simplex strategy will dynamically change the size and shape of the simplex as it 'rolls' toward minimum. When far away, it will increase its size in the direction to minimum, when it gets closer it will become smaller to better 'fit' into minimum. It will also change its direction if the shape of the multidimensional surface given by $efun$ has complex valleys and ridges. Due to the initially large size, the simplex will not 'feel' small dips int the surface and is not going to get stuck in one of these local minima. It works well for fitting noisy data. Alterntive nonlinear methods (such as Marquardt) are faster but prone to getting stuck in small local minima on the complex noise $\chi$ surface.


## Class Simplex

Initialize:

*Simplex(func,pars,dpars,max_iter = 10000, chi2_stop = 0, simp_stop = 1e-5, simp_stopMode = 3, <br>
apars = (), debug = 0, record = 0, silent = False)*
<p>
*efunc* - pointer to the 'black-box; error function to be minimized, it must be defined as follows<br>
<pre>
    def efunc(pars):
        pars[] - array of parameters to calculate the function
    def efunc(pars,x,y,any_other_data)
        The extra parameters above should be given in apars argument of Simplex class, 
        for above it will be the following tuple:
        apars = (x,y,any_other_data)       
</pre>

pars - array of initial parameters to calculate the function value (chi2)<br>
dpars - array of initial simplex size for each parameter. If 0 - the respective parameter is fixed<br>
apars - additional parameters to pass to *efunc*<br>
max_iter - maximum number of iterations (to avoid stalling forever)<br>
simp_stop_mode - The type of check to perform to end iterations and return<br>
<div style="margin-left: 50px;">
    simp_stop_mode = 0 # run through all max_iter iterations
    simp_stop_mode = 1 # stop when the efun returns value less than simp_stop
    simp_stop_mode = 2 # stop when the simplex size along chi2 (error values) reaches simp_stop
    simp_stop_mode = 3 # when the ratio of simplex size along chi2 over best chi2 is $<$ stop_chi2
</div>
simp_stop - the size of simplex below which to stop<br>

debug - store all steps in Simplex.debug_string as text. Print it after minimizing to see what was done.<br>
<div style="margin-left: 50px;">
    debug = 0 - do not store (saves time and space)<br>
    debug = 1 - store one record per iteration
    debug = 2 - store all steps in iteration
</div>
record - record simplex dynamics. 0 = do not record; 1 = record one per iteration; 2 - record all steps<br>
<div style="margin-left: 50px;">
    self.simp_record = [] - simplex verteces
    self.chi2_record = [] - chi2 for each vertex
    self.action_record = [] - what was done (text)
    self.iter_record = []   - iteration number
</div>
silent - if True, do not print iteation number and chi2 for each iteration during minimization<p>

Once initialized, run minimize function:<p>
    
    p,chi2best,i,simp,chi2 = Simplex.minimize()

The function returns tuple:<br>
p - best set of parameters<br>
chi2best - best chi2 achieved for this set<br>
simp - the last simplex (all verteces)<br>
chi2 - the array of chi2 for all verteces<p>

Usage:<p>
1. Define class simplex<br>
2. Run minimize function<p>

It is recommended to run it several times in a row, each time starting from previous best fit. This way you get to first minimum, then increase simplex size and run again from that point. Example:
<pre>
  pars = [2.,3.,...]     #efunc parameters
  dpars = [0.2,0.3,...]  #initial simplex size along each parameter
  simp_stop = 1e-2       #break itarative minimization when relative simplex size in chi2 is smaller than that
  iter_total = 0
  for i in range(8): 
      s = Simplex(efun,pars,dpars,simp_stop_mode=3,simp_stop = simp_stop...)
      pars,chi2,iteration, = s.minimize()  # change initial parameters to best fit
      iter_total += iteration # count total iterations for all runs
      simp_stop /=3 # in every next run to a smaller simplex size to be more precise
  print(f'Iteration = {iteration}, chi2 = {chi2}\nParameters = {pars}')
</pre>


    
    
    

          
            



In [ ]:
from math import *
import numpy as np

class Simplex:
    def __init__(self,efunc,pars,dpars,max_iter = 10000, chi2_stop = 1e-5, chi2_stop_mode = 0, apars = None, 
                 debug = 0, record = 0, silent = False):
        self.func = efunc                             # the function to be minimized, func(pars, apars)
        self.pars = np.array(pars, dtype = float)    # parameters to change []
        self.dpars = np.array(dpars, dtype = float)  # initial span for each parameter [], some can be zero to fix them
        self.apars = apars                           # additional parameters to pass to func as is, could be x and y arrays or whatever
        self.max_iter = max_iter                     # max number of iterations to avoid stalling
        self.chi2_stop_mode = chi2_stop_mode         # mode of stopping when chi2 reaches target
                                                     # 0 - do not use, run to max_iter iterations
                                                     # 1 - use absolute value of chi2 < stop_chi2
                                                     # 2 - when the simplex size along chi2 < stop_chi2
                                                     # 3 - when the ratio of simplex size along chi2 over best chi2 < stop_chi2
        self.chi2_stop = chi2_stop
        
        self.silent = silent   # set to False to not print iteration #
        self.debug = debug     # record all steps in minimization into  string for future inspection
        self.debug_string = "" # debug level = 0: dont record, 1 - 1 record per iteration, 2 - all steps in iteration
        self.record = record   # record simplex positions for future access, same 0,1,2
        self.simp_record = []
        self.chi2_record = []
        self.action_record = []
        self.iter_record = []
        return
        
    def init_simplex(self): # initialize simplex, i.., build initial simplex in parameter space
        self.ind = np.nonzero(self.dpars) # only change these! ind will have llist of indices that need to be changed
        self.p = self.pars[self.ind]      # array of pars to change
        self.d = self.dpars[self.ind]     # initial change values for them
        # build initial simplex
        self.simp = [self.p]      # first vertex is just initial values for fitting parameters
        for i in range(0,len(self.p)): # add oter verteces
            s = self.p.copy()
            s[i] += self.d[i]          # make new vertex by adding d-value to the parameter
            self.simp.append(s.copy()) # append new vertex
        self.simp = np.array(self.simp)         # size is N+1, where N is the number of parameters to change
        self.N = len(self.simp)
        return self.simp,self.N                 # returns simplex and number of verteces, if 1 then there are nothing to fit
    
    def func1(self,simp):  # call external function to calculate chi2 and retun chi2 value to be minimized
        p = self.pars.copy()       # make a copy of the initial parameters
        p[self.ind] = simp         # replace the changing parameters with new simplex vertex
        args = (p,) + self.apars   # make a list of p and append extra arguments in apars tuple, default apars = (), no extras
        chi2 = self.func(*args)    # call function with p and extra arguments (if apars was not an empy tuple, ())
        return chi2
    
    def get_chi2(self,simp):          # finds chi2 values for each simplex vertex (array)
        chi2 = np.zeros(len(simp))
        for i in range(len(chi2)):
            chi2[i] = self.func1(simp[i])   # simp[i] contains list of chnging parameters for vertex i
        return chi2

    def debug_record(self,txt,level = 0):
        if level >0 and level <= self.debug: self.debug_string = self.debug_string + txt 
        return
    
    def record_simp(self,simp,chi2,iteration,action,level = 0):
        if level > 0 and level <= self.record:
            self.simp_record.append(simp.copy())
            self.chi2_record.append(chi2.copy())
            self.iter_record.append(iteration)
            self.action_record.append(action)
        return
      
    def minimize(self):             # search minimum of functio, simplex motion strategy
        simp,n = self.init_simplex()
        if n <= 1: return           # only 1 vertex, no parameters to fit        
        simp = self.simp.copy()       
        for i in range(self.max_iter):
            chi2 = self.get_chi2(simp)
            self.record_simp(simp,chi2,i,f'Iteration {i} initialized',1)  # record this simplex
            self.debug_record(f'\n  ****** Iteration {i} initialized ******\nSimplex = \n{simp}\nchi2=\n{chi2}',1)
            # find best and worst verteces
            mi = np.argmin(chi2)
            ma = np.argmax(chi2)   
            if self.silent == False: print(f'\rIteration = {i}: {chi2[mi]:.5g}',end = '')
            
            if self.chi2_stop_mode == 1:
                if chi2[mi] < self.chi2_stop: break
            elif self.chi2_stop_mode == 2:
                if chi2[ma]-chi2[mi] < self.chi2_stop: break  
            elif self.chi2_stop_mode == 3:
                if chi2[mi] == 0: break
                if (chi2[ma]-chi2[mi])/chi2[mi] < self.chi2_stop: break
                    
            # reflect the worst point in respect of the best vertex
            simp_r = simp.copy()
            simp_r[ma] = 2*simp[mi] - simp[ma] # substitute the worst point with the new one
            chi2_r = self.get_chi2(simp_r)
            self.debug_record(f"\nreflect {ma} around {mi}\nSimplex = \n{simp_r}\nchi2=\n{chi2_r}",2)
            self.record_simp(simp_r,chi2_r,i,'reflect',2)
            
            if chi2_r[ma] < chi2[mi]: # the reflected point is better that previous best: increase simplex in this direction
                simp_i = simp.copy()
                simp_i[ma] = 3*simp[mi] - 2*simp[ma] # substitute the worst point with the new one
                chi2_i = self.get_chi2(simp_i)
                self.debug_record(f'\nreflected {ma} is better than best {mi}: expand\nSimplex = \n{simp_i}\nchi2=\n{chi2_i}',2)
                self.record_simp(simp_i,chi2_i,i,'reflected is much better, expand...',2)
                    
                if chi2_i[ma] < chi2_r[ma]: #if the doubly reflected point is better than singly - keep it and go to next iteration
                    self.debug_record(f' - Expanded!\nSimplex = \n{simp_i}\nchi2=\n{chi2_i}',2)
                    self.record_simp(simp_i,chi2_i,i,'Expanded!',2)
                    simp = simp_i
                    continue
                else: 
                    simp = simp_r
                    self.debug_record(f' Expanded is not better, revert\nSimplex = \n{simp_r}\nchi2=\n{chi2_r}',2)
                    self.record_simp(simp_r,chi2_r,i,'not better, revert!',2)
                    continue
                    
            elif chi2_r[ma] >= chi2[ma]: # the reflected point is even worse than the original: contract!
                simp[ma] = (simp[mi] + simp[ma])/2
                chi2 = self.get_chi2(simp)
                self.debug_record(f'\n  Reflection is worse, contract simplex!\nSimplex = \n{simp}\nchi2=\n{chi2}',2)
                self.record_simp(simp,chi2,i,'Reflection is worse: contract simplex',2)
                continue
            else: 
                simp = simp_r
                self.debug_record(f'\nPoint is better, keep reflected, Simplex = \n{simp}\nchi2=\n{chi2}',2)
                self.record_simp(simp,chi2,i,'Better, keep reflected',2)                
                
        chi2 = self.get_chi2(simp)
        mi = np.argmin(chi2)
        self.record_simp(simp,chi2,i,f'DONE! ',2)
        p = self.pars.copy()
        p[self.ind] = simp[mi]
        
        return p,chi2[mi],i,simp,chi2

# Visualize simplex process


## 1. Generate function to be minimized

This will use a bunch of inverted Gaussians added together to produce a nice valley surface with a single minimum

In [ ]:
#def func(p):
#    return (p[0]-10.)**2 + (p[1]-10.)**2

v = np.array([[-50.,-50.,20.],[-55.,-30.,20.],[-45.,-10.,20.],[-55.,10.,20.],[-50.,30.,20.],
              [-30.,40.,20.],[-10., 50.,20.],[10, 55, 20],[30,60,20],[50,60,20],
              [60,40,20],[60,20,20],[60,0,20],[55,-20,20],[55,-40,20],[50,-60,20],[50,-60,10]])
    
def func(p):
    z = 0
    p1, p2 = p[0], p[1]
    for i in range(len(v)):
        x,y,s = float(v[i][0]), float(v[i][1]), float(v[i][2])
        z = z - exp(-1/2*((p1-x)/s)**2) * exp(-1/2*((p2-y)/s)**2) * float(i)*2
    return z + 84.23628495 + 7.3751e-8


x = np.arange(-100.,100.,1.)
y = x.copy()
surf = np.zeros((len(x),len(y)))
for i in range(len(x)):
    for j in range(len(y)):
        surf[j][i] = func([x[i],y[j]])


## We will now plot that surface

In [ ]:
import matplotlib.pyplot as plt      # set of functions for drawing graphs

X, Y = np.meshgrid(x, y)
fig = plt.figure(figsize=(5,5),dpi=100)
#ax = fig.add_subplot(111, projection='3d')
ax = plt.axes(projection = '3d')
surfplot = ax.plot_surface(X, Y, surf, cmap='viridis', linewidth=0, antialiased=False, alpha=0.8)
zmin = -200 #np.min(surf) - 1
zmax = np.max(surf) + 1
ax.set_zlim(zmin,zmax)
ax.contourf(X, Y, surf, zdir='z', offset=zmin, cmap='viridis', alpha=0.8)

In [ ]:
from math import *
import matplotlib.pyplot as plt      # set of functions for drawing graphs
from matplotlib.animation import FuncAnimation  # set of functions to create animations using set of plots
from IPython.display import HTML, display, clear_output     # to export instructions in html format (inline Jupiter output)
import matplotlib

matplotlib.rcParams['animation.embed_limit'] = 500

par = [-70.4321,-75.12123]
dpar = [27.1212,11.23451]

s = Simplex(func,par,dpar,max_iter = 200,chi2_stop_mode = 2,chi2_stop = 1e-5,record = 2,debug = 2,apars = ()) 
s.minimize()

        
fig = plt.figure(figsize=(5,5),dpi=100)
ax = plt.axes()
#ax.set_aspect('equal')
imshow = plt.imshow(surf, extent = (x[0],x[-1],y[0],y[-1]), origin='lower', cmap='viridis') # 2D color map of wavefunction
#plt.plot(v[:,0],v[:,1]) # plot path
X, Y = np.meshgrid(x, y)
contmax=np.max(surf)
contstep = contmax/50
plt.contour(X, Y, surf, levels=np.arange(contstep/2,contmax,contstep), colors='white', linestyles='solid',linewidths = 0.3)

line_simp, = plt.plot([-20,],[-20],color = 'red')
#scat, = plt.scatter([0,0,0],[0,0,0],c = [0,.5,1],cmap = 
text_handle = plt.text(-90,90,'Iter 0')
text_action = plt.text(-90,80,'Action')
text_extra = plt.text(-90,60,'.',fontsize=8)

def plotsimp(simp,chi2,i,action,frame):
    #global seq2
    #seq2 = seq2 + f'{frame}: i{i}, {simp[0]}, {simp[1]}, {simp[2]} \n'
    simp1 = np.append(simp,[simp[0]],axis=0)
    line_simp.set_data(simp1[:,0],simp1[:,1])
    text_handle.set_text(f'{frame}:Iter {i}: chi2 = {np.min(chi2):.5g}')
    text_action.set_text(action)
    text_extra.set_text(f'{simp}')
    #display.display(fig)
    #time.sleep(.01)
    #display.clear_output(wait=True)
    
def update(i):
    print(f'\rFrame {i} of {len(s.simp_record)}: chi2={s.chi2_record[i]}       ',end = '')
    plotsimp(s.simp_record[i],s.chi2_record[i],s.iter_record[i],s.action_record[i],i)
    
ani = FuncAnimation(fig, update, frames=len(s.simp_record), interval=200, blit=False) # setting blit=True does not replot map!
#ani.fargs = (ani,)
plt.close()

save_mp4 = False
show_html = True
if save_mp4 == True:
    name = 'simplex1'
    print(f'\nMaking MP4 animation: {name}.mp4')
    from matplotlib.animation import writers
    Writer = writers['ffmpeg']
    writer = Writer(metadata=dict(artist='Sergei Savikhin', genre = 'Minimum Search', title = 'Simplex method'),fps = 5)
    ani.save(name+'.mp4', writer = writer)
    print('done')

if show_html == True:
    print('\nMaking jshtml animation:\n')
    anim = ani.to_jshtml()
    print('done')
    display(HTML(anim))


# Fit some data

Here we generate a ouble gaussian, add noise to it. Each Gaussian is defined by 3 parameters - amplitude, position, and width. For the sum of two Gaussians we will have 6 free parameters. The parameter's space is 6-dimensional, and simplex will have 7 verteces.

In [ ]:
from math import *
import matplotlib.pyplot as plt      # set of functions for drawing graphs
from matplotlib.animation import FuncAnimation  # set of functions to create animations using set of plots
from IPython.display import HTML, display, clear_output     # to export instructions in html format (inline Jupiter output)
import matplotlib

matplotlib.rcParams['animation.embed_limit'] = 100

def func(p,x): # generate sum of gaussians along x-axis (array). p = [pos1,width1,amp1, pos2,width2,amp2, ...]
    pos, width, amp = [], [], []   # the number of parameters /3 = number of gaussians to add
    for i in range(0,len(p),3):
        pos.append(p[i]);   width.append(p[i+1]); amp.append(p[i+2]) # to make the use more transparent
    y = np.zeros(len(x)) # will collect the sum here
    for i in range(len(pos)):
        y = y + amp[i]*np.exp(-1/2*((pos[i]-x)/width[i])**2)
    return y

last_chi2 = 1.e30  # this remembers the chi2 for the last call to this function
def efunc(p,x,y,line = None,txt = None): # calculate chi2 for given set of parameters, called by simplex
    global last_chi2
    # calculate theoretical function
    yy = func(p,x)
    chi2 = sqrt(np.sum((yy - y)**2)/len(x))
    # plot it!
    if chi2 < last_chi2:  # the chi2 improved from the last plot, replot the fit (animation), if line and txt are given
        if line != None: line.set_data(x,yy)
        if txt != None: txt.set_text(f'chi2 = {chi2}    ')
        display(plt.gcf())  # this and the next line replot in display in real time (we do not use animation class here)
        clear_output(wait=True)
        last_chi2 = chi2
    return chi2

x = np.arange(0.,100.,1.)

y = func(p = [30.,5.,2.,  60.,10.,1.], x = x) # get theoretical curve
y_exp = np.random.normal(loc=y,scale=.1,size = (len(x))) # add random noise to mimic experiment, this will be the data to fit

# Create plot
fig = plt.figure(figsize=(5,5),dpi=100)
ax = plt.axes()
plt.scatter(x,y_exp,s=20,c='red') # use scatterplot to show the experimental data (with noise)
plt.plot(x,y,color = 'green')     # plot the original data without noise
plt.xlim(np.min(x),np.max(x))
ymi, yma = np.min(y_exp),np.max(y_exp)
ad = (yma-ymi)/10
plt.ylim(ymi-ad,yma+ad*2) # redefine y-limits 
line_fit, = plt.plot([],[],color = 'blue') # this will be the current best fit, replotted in efunc
text = plt.text(0,yma+ad,'')               # this is the placeholder for some text printed on plot in efunc

p = [29.,4.,2.1,  61., 11., 0.9] # if I make reasonable initial guess
#p = [25.,4.,1.5,  55., 8., 0.9]  # bad guess
#p = [20.,14.,1.5,  75., 5., 0.9]  # very bad guess
dp = [2.,.5,.3,   5.,  .4, .1]    # initial parameter changes (initial simplex size)
plt.plot(x,func(p,x),color = 'gray')  # plot the initial guess
iter_total = 0
chi2_stop = 1e-2
for mumu in range(8): # we will run simplex several times back-to-back to get out of local minima (if any)
    s = Simplex(efunc,p,dp,apars = (x,y_exp,line_fit,text),max_iter = 1000,
                chi2_stop_mode = 3, chi2_stop = chi2_stop,silent = True)
    p,chi2best,iterations,simp,chi2 = s.minimize() # we will set 'initial' parameters to the best fit in this run
    iter_total += iterations
    chi2_stop /= 3. # decrease stopping simplex size for each next iteration as we are getting closer


print(f'\nNoise added: chi2 = {efunc([30.,5.,2.,60.,10.,1.],x,y_exp)}')
print(f'best pars: {p}')
print(f'chi2 = {chi2best}')
print(f'Iter_total = {iter_total}')
